In [1]:
##!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage firebase-admin
!pip install langchain_community
!pip install --upgrade google-cloud-aiplatform
!pip install google-cloud-storage
!pip install langchain_community
!pip install langchain
!pip install pymuPDF
!pip install -U langchain-google-vertexai
!gcloud auth application-default login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00


In [6]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=4SbAcGF8fDPdHh7VgCGKuPsxmTceHG&prompt=consent&token_usage=remote&access_type=offline&code_challenge=Y6gjxga06fTNtr7CdtwE0TehfuMkqjL_r-DshYNsXpM&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8wcEB5LahIkSBPZikPXvOcziI3b_ShTtW1AftGBr2_5rfz_cPvITWEQ5pZF6tEYrg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [11]:
import fitz  # PyMuPDF

# Open the PDF document
pdf_document = "FINALIZADO_Aquivo 1_Arquivos_SS_erro_SS_ASSINADA_ACA 5056442_campo_SS_assinada_0.pdf"
pdf = fitz.open(pdf_document)

# Read and store the content of each page
pages_content = []
for page_num in range(len(pdf)):
    page = pdf[page_num]
    pages_content.append(page.get_text())
print(pages_content[3][:200])

SOLICITAÇÃO DE SERVIÇO – CONTRATO CAPACIDADE  CW2413024
ANEXO À SOLICITAÇÃO DE CAPACIDADE Nº 0015/2023 – (Ativação – Fase 1 – Capacidade Inicial) - Formulario de Solicitação de Serviço-FSS
 OBSERVAÇÕE


In [5]:
len(pages_content)

20

In [7]:
textsi_1 = """você é um agente que vai ler documentos em pdf e vai ler os anexos, neste anexos terá um tabela quero que você leia as linhas e retorno o json com:
item: <numero do item>,
cliente: <separar em um novo dicionário cada item encontrado: da empresa contratante>
      { razaosocial:<razão social ou nome>,
        endereco:<endereço rua numero e bairro>,
        cidade:<cidade>,
        cep:<cep>,
        uf:<uf>,
        cnpj:<cnpj>,
        inscricaoestadual:<inscrição estadual>,
        inscricaomunicipal:<inscrição municipal ou insc municipal>,
        telefone:<telefone deve começar por +55>,
        email:<email>
      },
Velocidade: <velocidade do produto>
enderecoA: <Endereço Ponta A ou endereço da instalação ou endereço coletora>,
enderecoB: <Endereço Ponta B ou endereço concentrador>,
valorInstalacao : <Valor instalação ou ativação, valor ativação, valor instalação>
valorMensal: <Valor Mensal ou MRC, Valor MRC, Valor Mensal, Valor Mensalização>"""

In [10]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.preview.generative_models as generative_models

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}


In [13]:
def generate(document):
  vertexai.init(project="vtal-apiviabilidade-prd", location="southamerica-east1")
  model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=[textsi_1]
  )
  responses = model.generate_content(
      [document],
      generation_config=generation_config,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")


In [14]:

#for page in pages_content:
#  generate(page)
generate(pages_content[2])

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


```json
[
  {
    "numero do item": "42",
    "Dados cadastrais de cliente": {
      "nome": "TIM S.A.",
      "razao social": "TIM S.A.",
      "CNPJ": "02.421.421/0001-11",
      "Endereço": "Av. João de Cabral de Melo Neto, 850 -  CEO - Torre Sul - 5º andar - Barra da Tijuca",
      "cidade": "Rio de Janeiro",
      "cep": "22775 - 057",
      "uf": "RJ",
      "inscrição estadual": "86.092.08-5",
      "inscrição municipal": null,
      "telefone": "0261388-3 +55 21 98113-1783",
      "email": "bamendonca@timbrasil.com.br"
    },
    "Numero pedido": null,
    "Tipo de solicitação": "Ativação",
    "Produto": "FTTSITE",
    "Velocidade produto": "10 Gbps",
    "Endereço Ponta A (ou endereço da instalação)": "VIA EXPRESSA MARCIO DE SOUZA RESENDE, 341, 0, CENTRO",
    "Endereço Ponta B": "AVENIDA OSVALDO WOLGARINE , S/N,  COM RUA CRUZ DE MALTA, QD. 12, LT. 01, , BAIRRO SETOR RODOVIARIO",
    "Valor instalação (ou ativação, valor ativação, valor instalação, etc)": "R$ 91.505,60",
    

In [ ]:
# prompt: tem como pegar a saida do json e colocar num data frame

import json
import pandas as pd

# Assuming 'json_output' is the string containing the JSON output from the model
json_output = response.text

# Load the JSON string into a Python dictionary
data_dict = json.loads(json_output)

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data_dict)
